In [ ]:
import nltk
import numpy as np
import pandas as pd
import string
import re
import keras
import os
from pandas import Series
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from tensorflow.keras.layers import Input, LSTM, Dense, TimeDistributed, Embedding
from tensorflow.keras.models import Model, load_model
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

# import warnings
# warnings.filterwarnings("ignore")

In [ ]:
data1 = pd.read_excel('QA.1.xlsx')
data2 = pd.read_excel('QA.2.xlsx')
data3 = pd.read_excel('QA.3.xlsx')
data4 = pd.read_excel('QA_4.xlsx')
data5 = pd.read_excel('QA_5.xlsx')
data6 = pd.read_excel('QA_6.xlsx')
data = pd.concat([data1, data2, data3, data4, data5, data6])
#data.to_csv('Compiled.csv')

In [ ]:
#list(data4['Answer'])[0]
#list(data4['Answer'])[0].replace('\n',' ').split('.')

In [ ]:
# data1.head()
# data2.head()
# data3.head()
# data4.head()
# data5.head()
# data6.head()
data.head()

In [ ]:
data.shape

In [ ]:
nltk.tokenize.sent_tokenize((data[data['Question'] == 'What is meant by selection bias?']['Answer'])[0])

In [ ]:
#nltk.tokenize.sent_tokenize(list(data['Answer'])[0])

In [ ]:
sdata = (data.set_index(['Question'])
         .apply(lambda x: x.str.replace('\n',' ').str.split('.').explode())
         .reset_index())
sdata['Answer'].replace('', np.nan, inplace = True)
csdata = sdata.dropna().reset_index(drop = True)
df = csdata.loc[csdata['Answer'].str.len() > 3].reset_index(drop = True)

In [ ]:
def generate_root(words):
    lemmatizer = WordNetLemmatizer()
    lemmatized = [lemmatizer.lemmatize(word.lower(), pos = 'n') for word in words]
    return lemmatized
def preprocess(pr):
    #Remove Punctuations
    no_punc = ''.join([character for character in pr if character not in string.punctuation])  
    #Tokenize the PRT
    tokenized = nltk.word_tokenize(no_punc.lower()) 
    #Remove Stop Words
    stop_words = set(stopwords.words('english'))
    no_stop = [word.lower() for word in tokenized if word.lower() not in stop_words]
    no_numeric = [word for word in no_stop if not any(num.isdigit() for num in word)]
    #Generating the root words of the tokens
    root_word = generate_root(no_numeric)        
    return root_word

def preprocessQ(pr):
    #Remove Punctuations from Q only
    no_punc = ''.join([character for character in pr if character not in string.punctuation])
    return no_punc

In [ ]:
def remove_puncs(text):
    no_punc = ''.join([character for character in text if character not in string.punctuation])
    return no_punc
df['CleanA'] = df['Answer'].apply(lambda x: remove_puncs(x))
#df['CleanQ'] = df['Question'].apply(lambda x: remove_puncs(x))

def tokenize(pr):
    tokenized = nltk.word_tokenize(pr.lower())
    return tokenized
df['TokenizedA'] = df['CleanA'].apply(lambda x: tokenize(x.lower()))
#df['TokenizedQ'] = df['CleanQ'].apply(lambda x: tokenize(x.lower()))

def remove_stop(text):
    stop_words = set(stopwords.words('english'))
    no_stop = [word.lower() for word in text if word.lower() not in stop_words]
    no_numeric = [word for word in no_stop if not any(num.isdigit() for num in word)] #We may want to keep numbers
    #return no_stop
    return no_numeric
df['No_StopA'] = df['TokenizedA'].apply(lambda x: remove_stop(x))
# df['No_StopQ'] = df['TokenizedQ'].apply(lambda x: remove_stop(x))


def generate_root(words):
    lemmatizer = WordNetLemmatizer()
    lemmatized = [lemmatizer.lemmatize(word.lower(), pos = 'n') for word in words]
    return lemmatized
def rooting(sentence):
    root_word = generate_root(sentence)        
    return root_word
df['RootedA'] = df['No_StopA'].apply(lambda x: rooting(x))
df['FilteredA'] = df['RootedA'].apply(lambda x: (' ').join(x))
# df['RootedQ'] = df['No_StopQ'].apply(lambda x: rooting(x))
# df['FilteredQ'] = df['RootedQ'].apply(lambda x: (' ').join(x))

In [ ]:
# finaldf = df[['Question', 'Filtered']]
# finaldf = finaldf.loc[finaldf['Filtered'].str.len() > 3].reset_index(drop = True)
finaldf = df[['Question', 'FilteredA']]
finaldf = finaldf.loc[finaldf['FilteredA'].str.len() > 3].reset_index(drop = True)
pairs = list(zip(finaldf.Question, finaldf.FilteredA))
len(pairs)

In [ ]:
#Medium's Sample Guide
input_docs = list(finaldf['FilteredQ']) #List of all Questions
target_docs = [] #list(finaldf['Filtered']) #List of all Filtered Answers
input_tokens = set() #Unique Vocabulary in Questions
target_tokens = set() #Unique Vocabulary in Answers
for line in pairs:
    input_doc, target_doc = line[0], line[1]
    # Splitting words from punctuation  
    target_doc = " ".join(re.findall(r"[\w']+|[^\s\w]", target_doc))
    # Redefine target_doc below and append it to target_docs
    target_doc = '<START> ' + target_doc + ' <END>'
    target_docs.append(target_doc)

    # Now we split up each sentence into words and add each unique word to our vocabulary set
    for token in re.findall(r"[\w']+|[^\s\w]", input_doc):
        if token not in input_tokens:
            input_tokens.add(token)
    for token in target_doc.split():
        if token not in target_tokens:
            target_tokens.add(token)

input_tokens = sorted(list(input_tokens))
target_tokens = sorted(list(target_tokens))
num_encoder_tokens = len(input_tokens)
num_decoder_tokens = len(target_tokens)

input_features_dict = dict(
    [(token, i) for i, token in enumerate(input_tokens)])
target_features_dict = dict(
    [(token, i) for i, token in enumerate(target_tokens)])
reverse_input_features_dict = dict(
    (i, token) for token, i in input_features_dict.items())
reverse_target_features_dict = dict(
    (i, token) for token, i in target_features_dict.items())

#Maximum length of sentences in input and target documents
max_encoder_seq_length = max([len(re.findall(r"[\w']+|[^\s\w]", input_doc)) for input_doc in input_docs])
max_decoder_seq_length = max([len(re.findall(r"[\w']+|[^\s\w]", target_doc)) for target_doc in target_docs])
encoder_input_data = np.zeros((len(input_docs), max_encoder_seq_length, num_encoder_tokens), dtype = 'float32')
decoder_input_data = np.zeros((len(input_docs), max_decoder_seq_length, num_decoder_tokens), dtype = 'float32')
decoder_target_data = np.zeros((len(input_docs), max_decoder_seq_length, num_decoder_tokens), dtype = 'float32')
for line, (input_doc, target_doc) in enumerate(zip(input_docs, target_docs)):
    for timestep, token in enumerate(re.findall(r"[\w']+|[^\s\w]", input_doc)):
        #Assign 1. for the current line, timestep, & word in encoder_input_data
        encoder_input_data[line, timestep, input_features_dict[token]] = 1.
    for timestep, token in enumerate(target_doc.split()):
        decoder_input_data[line, timestep, target_features_dict[token]] = 1.
        if timestep > 0:
            decoder_target_data[line, timestep - 1, target_features_dict[token]] = 1.

In [ ]:
encoder_input_text = list(finaldf['Question'])
def addtags(decoder_input_sentence):
    bt = "<BOS> "
    et = " <EOS>"
    final_target = [bt + text + et for text in decoder_input_sentence] 
    return final_target
decoder_input_text = addtags(finaldf['FilteredA'])

In [ ]:
def vocab_creater(text_lists, VOCAB_SIZE):
    tokenizer = Tokenizer(num_words = VOCAB_SIZE)
    tokenizer.fit_on_texts(text_lists)
    dictionary = tokenizer.word_index
    word2idx = {}
    idx2word = {}
    for k, v in dictionary.items():
        if v < VOCAB_SIZE:
            word2idx[k] = v
            idx2word[v] = k
        if v >= VOCAB_SIZE-1:
            continue      
    return word2idx, idx2word
word2idx, idx2word = vocab_creater(encoder_input_text + decoder_input_text, 10000)

In [ ]:
word2idx

In [ ]:
idx2word

In [ ]:
eword2idx, eidx2word = vocab_creater(encoder_input_text, 10000)
num_unique_encoder = len(eword2idx)
dword2idx, didx2word = vocab_creater(decoder_input_text, 10000)
num_unique_decoder = len(dword2idx)

In [ ]:
num_unique_encoder

In [ ]:
def text2seq(encoder_text, decoder_text, VOCAB_SIZE):
    tokenizer = Tokenizer(num_words = VOCAB_SIZE)
    tokenizer.fit_on_texts(encoder_text + decoder_text)
    encoder_sequences = tokenizer.texts_to_sequences(encoder_text)
    decoder_sequences = tokenizer.texts_to_sequences(decoder_text)
    return encoder_sequences, decoder_sequences

encoder_sequences, decoder_sequences = text2seq(encoder_input_text, decoder_input_text, 10000) 

In [ ]:
encoder_sequences

In [ ]:
decoder_sequences

In [ ]:
max_encoder_seq_length = max([len(re.findall(r"[\w']+|[^\s\w]", inputs)) for inputs in encoder_input_text])
max_decoder_seq_length = max([len(re.findall(r"[\w']+|[^\s\w]", inputs)) for inputs in decoder_input_text])
if max_encoder_seq_length > max_decoder_seq_length:
    MAX_LEN = max_encoder_seq_length
else:
    MAX_LEN = max_decoder_seq_length
    
def padding(encoder_sequences, decoder_sequences, MAX_LEN):
    encoder_input_data = pad_sequences(encoder_sequences, maxlen = max_encoder_seq_length, dtype = 'int32', padding = 'post', truncating = 'post')
    decoder_input_data = pad_sequences(decoder_sequences, maxlen = max_decoder_seq_length, dtype = 'int32', padding = 'post', truncating = 'post')
    return encoder_input_data, decoder_input_data
encoder_input_data, decoder_input_data = padding(encoder_sequences, decoder_sequences, MAX_LEN)
decoder_target_data = np.zeros((len(encoder_input_text), max_decoder_seq_length), dtype = 'int32')

In [ ]:
res = max(encoder_input_text, key = len) 
res

In [ ]:
encoder_input_data.shape

In [ ]:
decoder_input_data.shape

In [ ]:
encoder_input_layer = Input(shape = (max_encoder_seq_length, ))
decoder_input_layer = Input(shape = (max_decoder_seq_length, ))

In [ ]:
def glove_100d_dictionary(): #Creating the embedding index with GloVe's 100d.
    embeddings_index = {}
    data = open('glove.6B.100d.txt', encoding = 'utf8')
    for line in f:
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype = 'float32')
        embeddings_index[word] = coefs
    data.close()
    return embeddings_index
embeddings_index = glove_100d_dictionary()

In [ ]:
def embedding_matrix_creater(embedding_dimension): #Creating the embedding matrix with the embedding index.
    embedding_matrix = np.zeros((len(word2idx) + 1, embedding_dimension))
    for word, i in word2idx.items():
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector
    return embedding_matrix
embedding_matrix = embedding_matrix_creater(100)

In [ ]:
def embedding_layer_creater(VOCAB_SIZE, EMBEDDING_DIM, MAX_LEN, embedding_matrix): #Creating the embedding layer.
    embedding_layer = Embedding(input_dim = VOCAB_SIZE, 
                                output_dim = EMBEDDING_DIM,
                                input_length = MAX_LEN,
                                weights = [embedding_matrix],
                                trainable = False)
    return embedding_layer
embedding_layer = embedding_layer_creater(3041, 100, MAX_LEN, embedding_matrix)

In [ ]:
def data_spliter(encoder_input_data, decoder_input_data, test_size1 = 0.2, test_size2 = 0.3):
    encoder_train, encoder_test, decoder_train, decoder_test = train_test_split(encoder_input_data, decoder_input_data, test_size = test_size1)
    encoder_train, encoder_val, decoder_train, decoder_val = train_test_split(encoder_train, decoder_train, test_size = test_size2)
    return encoder_train, encoder_val, encoder_test, decoder_train, decoder_val, decoder_test
encoder_train, encoder_val, encoder_test, decoder_train, decoder_val, decoder_test = data_spliter(encoder_input_data, decoder_input_data)

print(encoder_train.shape)
print(encoder_val.shape)
print(encoder_test.shape)
print(decoder_train.shape)
print(decoder_val.shape)
print(decoder_test.shape)

In [ ]:
#encoder_inputs = Input(shape = (MAX_LEN, ), dtype = 'int32',)
encoder_inputs = Input(shape=(num_unique_encoder, ), dtype = 'int32',)
#encoder_inputs = Input(shape = (None, num_encoder_tokens))
#encoder_embedding = embedding_layer(encoder_inputs)
encoder_LSTM = LSTM(256, return_state = True)
#encoder_outputs, state_h, state_c = encoder_LSTM(encoder_embedding)
encoder_outputs, state_h, state_c = encoder_LSTM(encoder_inputs)

#decoder_inputs = Input(shape = (MAX_LEN, ), dtype = 'int32',)
decoder_inputs = Input(shape = (num_unique_decoder, ), dtype = 'int32',)
#decoder_inputs = Input(shape = (None, num_decoder_tokens))
#decoder_embedding = embedding_layer(decoder_inputs)
decoder_LSTM = LSTM(256, return_state = True, return_sequences = True)
#dense_layer = Dense(VOCAB_SIZE, activation = 'softmax')
#decoder_dense_layer = Dense(num_unique_decoder, activation = 'relu')
decoder_dense_layer = Dense(num_unique_decoder, activation = 'softmax')
decoder_outputs, _, _ = decoder_LSTM(decoder_embedding, initial_state = [state_h, state_c])
#decoder_outputs = decoder_dense_layer(decoder_outputs)
outputs = TimeDistributed(Dense(10000, activation = 'softmax'))(decoder_outputs)

model = Model([encoder_inputs, decoder_inputs], outputs)

In [ ]:
#Model
model = Model([encoder_inputs, decoder_inputs], outputs)
#Compiling
model.compile(optimizer = 'rmsprop', loss = 'categorical_crossentropy', metrics = ['accuracy'], sample_weight_mode = 'temporal')
#model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'], sample_weight_mode = 'temporal')

#Fitting
model.fit([encoder_train, decoder_train], outputs, batch_size = 64, epochs = 60, shuffle = True)
#model.fit([encoder_train, decoder_train], outputs, batch_size = 32, epochs = 60, shuffle = True)


In [ ]:
model.save('model.h5')

In [ ]:
encoder_inputs = training_model.input[0]
encoder_outputs, state_h_enc, state_c_enc = training_model.layers[2].output
encoder_states = [state_h_enc, state_c_enc]
encoder_model = Model(encoder_inputs, encoder_states)

In [ ]:
def decode_response(test_input):
    states_value = encoder_model.predict(test_input)
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    target_seq[0, 0, target_features_dict['<BOS>']] = 1.
    decoded_sentence = ''
    stop_condition = False
    while not stop_condition:
        output_tokens, hidden_state, cell_state = decoder_model.predict([target_seq] + states_value)
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_token = reverse_target_features_dict[sampled_token_index]
        decoded_sentence += " " + sampled_token
        if (sampled_token == '<EOS>' or len(decoded_sentence) > max_decoder_seq_length):
            stop_condition = True
            target_seq = np.zeros((1, 1, num_decoder_tokens))
            target_seq[0, 0, sampled_token_index] = 1.
            states_value = [hidden_state, cell_state]
    return decoded_sentence
class ans:
    def string_to_matrix(self, user_input):
        tokens = preprocessQ(user_input)
        user_input_matrix = np.zeros(
          (1, max_encoder_seq_length, num_encoder_tokens), dtype = 'float32')
        for timestep, token in enumerate(tokens):
            if token in input_features_dict:
                user_input_matrix[0, timestep, input_features_dict[token]] = 1.
        return user_input_matrix
    def generate_response(self, user_input):
        input_matrix = self.string_to_matrix(user_input)
        chatbot_response = decode_response(input_matrix)
        chatbot_response = chatbot_response.replace("<BOS>",'')
        chatbot_response = chatbot_response.replace("<EOS>",'')
        return chatbot_response

In [ ]:
Questiontoask = 'What is meant by selection bias?'
answertoquestion = ans().generate_response(Questiontoask)
print('Input: ' + Questiontoask)
print('Output: ' + answertoquestion)